In [58]:
import openai
from openai import OpenAI

In [59]:
#To wrap text
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [60]:
from typing import List, Dict
history: List[Dict[str, str]] = []
#Defining a class for all agents
class Agent:

    def __init__(self, name: str, system_prompt: str):
        self.name = name
        self.system_prompt = system_prompt.strip()
        self.client = OpenAI(api_key="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", base_url="https://api.groq.com/openai/v1")

    def _format_prompt(self, user_msg: str) -> str:

        messages = [{"role": "system", "content": self.system_prompt}]
        messages.extend(history)
        messages.append({"role": "user", "content": user_msg})

        return messages


    def respond(self, user_msg: str, **gen_kwargs) -> str:

        prompt = self._format_prompt(user_msg)
        response = self.client.chat.completions.create(model="mistral-saba-24b", messages=prompt, temperature=0.7, max_tokens=200)
        actualres = response.choices[0].message.content
        actualres = actualres.strip()
        history.append({"role": "user", "content": user_msg})
        history.append({"role": "assistant", "content": actualres})
        return actualres


In [63]:
#The prompts

DEFENSE = """
You are an attorney who must argue for a defendant's innocence in a case and defend him from accusations from the prosecution.
Goals:
• Protect the constitutional rights of the defendant.
• Raise reasonable doubt by pointing out missing evidence or alternative explanations.
• Be concise, persuasive and assertive
• When citing precedent: give short case name + year (e.g., *Miranda v. Arizona* (1966)).
• Do not fabricate evidence; admit uncertainty when required.
Finish your ENTIRE response within 100 words. Do not stop mid sentence.
Start EVERY response with 'Defense : '
"""

PROS = """
You are an attorney who must argue for the plaintiff and logically assert that the defence is guilty.
Goals:
• Present the strongest good‑faith case against the accused.
• Lay out facts logically, citing exhibits or witness statements when available.
• Anticipate and rebut common defense arguments.
• Be assertive and confident
• Concede points when ethically required.
Finish your ENTIRE response within 100 words. Do not stop mid sentence.
Start EVERY response with 'Prosecution : '
"""
JUDGE = """
You are the honourable judge of a court who is hearing a case between a plaintiff and a defendant.
Goals:
• Keenly analyse the points raised by both the prosecutor and the defendant in pertinence to the case.
• Then, make the final verdict. Decide whether the defendant is guilty or not based on the arguments shared from both sides.
Style:
• Be formal, authoritative, impartial, and calm.
Ethics:
• Remain completely unbiased, justice is fair.
• Derive conclusions solely based on the facts that have been presented.

Finish your ENTIRE response within 100 words. Do not stop mid sentence.
Start EVERY response with 'Judge : '
"""

DEFENDANT = """
You are the defendant in a case, and are being held under trial.
Goals:
• Testify only if it provides a strategic advantage. Do not say anything that will incriminate you.
• You are allowed to deny the crime, provide an alibi or justify your actions. Decide which is most appropriate. If it is most appropriate to remain silent, remain silent.
• If there is no way out, you may concede to having committed a lesser offence.
Keep your entire response under 50 words. Do not stop mid sentence.
Start EVERY response with 'Defendant : '
"""

PLAIN = """
You are the plaintiff in a case and have initiated a trial against a defendant.
Goals:
• Testify in order to emphasize on the defendant's wrongdoing, only if you feel that the defence has made an argument that is unreasonable.
• Do not interrupt court proceedings unless absolutely necessary, and leave the majority of the talking to your lawyer.
Keep your entire response under 50 words. Do not stop mid sentence.
Start EVERY response with 'Plaintiff : '
"""
WITNESS = """
You are a witness to an event being discussed in a courtroom trial.
Goals:
• Be clear and concise with your speech.
• Only answer if you have been asked a question by the Defence or the Prosecution (in your prompt). Do not raise points of your own.
• Be completely truthful and stick to your alibi.
Keep your entire response under 75 words. Do not stop mid sentence.
Start EVERY response with 'Witness : $'
"""

#Declaration of the five agents. Witnesses are declared only conditionally.

defense = Agent("Defense", DEFENSE)
prosecution = Agent("Prosecution", PROS)
judge = Agent("Judge", JUDGE)
defendant = Agent("Defendant", DEFENDANT)
plaintiff = Agent("Plaintiff", PLAIN)

case_background = "With Criminal Appeal No.227 of 1997 K. Sabharwal, J. These appeals by special leave challenge the judgment of the High Court by which the companyviction of the appellants for offence under Section 201 of the Indian Penal Code has been upheld. The appellants in Criminal Appeal No.151 of 1997 are family members of Baldev Singh. Appellant Nos. 3 and 5 are brothers of Baldev Singh and appellant No.4 is wife of appellant No.3 whereas appellant No.6 is wife of appellant No.5. Appellant No.7 is sister of Baldev Singh. Baldev Singh is son of appellant No.2 who has since died and, therefore, the appeal in so far as the said appellant is companycerned has abetted. The sole appellant, Harjap Singh, in Criminal Appellant No.227 is family friend of Baldev Singh. The case of the prosecution in brief is as under Baldev Singh was married with Manmohan Kaur in the year 1979. It is alleged that Baldev Singh was harassing Manmohan Kaur for number bringing sufficient amount of dowry. She visited house of her father in village Nangal Thindal on 21st June, 1985 and told him that a demand of Rs.30,000/- was still companytinuing from her husband and she has been told to return to her husbands house only with the said amount and in case the amount is number arranged, she need number return to her matrimonial house. Her father, PW4  Harbans Singh , however, sent back his daughter to her matrimonial house with assurance that he would soon visit the house of her in-laws and settle the matter there. Manmohan Kaur left behind her three years old daughter at her parents house. On 22nd June at about 2.35 p.m., Baldev Singh came to the house of his father-in-law and informed him that Manmohan Kaur was missing from the house since early morning that day. Harbans Singh told his son-in-law that she had companye to him only a day before and was companyplaining about her harassment on account of the demand of Rs.30,000/- made by him, his parents, sisters, sisters-in-law and brothers as dowry and asked his son-in-law to have a thorough search of Manmohan Kaur. Harbans Singh also asked his son-in-law to send information to him immediately when she is found and on his own he went out along with his relatives to search his daughter. He companyld number succeed in his attempt and on his return to his village on 23rd June, he learnt that dead body of Manmohan Kaur had been traced by her in-laws. Immediately, Harbans Singh rushed to the house of the in-laws of Manmohan Kaur. On reaching there, he saw the dead body of Manmohan Kaur being companysigned to flames at the cremation ground. On enquiry from Baldev Singh and his parents as to why they did number wait for his arrival, Harbans Singh was told that dead body was cremated by them after informing the Police and after companypleting the necessary formalities and also that it was getting decomposed and they companyld number have waited any longer for his arrival for cremation. On 24th June, Harbans Singh filed a written application  Exhibit PD  with the Senior Superintendent of Police, Hoshiarpur expressing his doubt that his daughter has been murdered by her in-laws for number bringing sufficient dowry. It was mentioned therein that he was suspecting the appellants besides Baldev Singh and his father. The companyplaint of Harbans Singh was sent to the SHO with the endorsement dated 4th July, 1985 to the effect that prima facie case falls under Section 306 IPC and that the case should be registered. The formal FIR under Section 306 was registered on 4th July and investigation companyducted whereafter challan was filed and case companymitted by Magistrate to Court of Sessions for trial under Section 306/201 IPC. Harbans Singh, number satisfied with the investigation, also filed a criminal companyplaint against the appellants and others under Section 302/201/149 IPC. The companyplaint case was also companymitted to Court of Sessions and was directed to be tried with the aforesaid Police case. Both the cases were companysolidated. The Sessions Court acquitted all the accused of offence under Section 302 as also of offence under Section 302/149 IPC. For offence under Section 306, Baldev Singh was held guilty. The appellants were acquitted of charge under Section 306. They were, however, found guilty of offence punishable under Section 201 IPC and Rigorous Imprisonment for one year was imposed on each of them besides fine. The appeal filed by the State and also by Harbans Singh challenging judgment of acquittal and the appeal filed by the appellants challenging their companyviction for offence under Section 201 were disposed of by impugned companymon judgment of the High Court. All the appeals have been dismissed. Thus, the acquittal of the appellants for offence under Section 306 has been companyfirmed. The acquittal of the appellants for offence under Section 306 IPC has attained finality. The companyviction of Baldev Singh for offence under Section 306 IPC has also attained finality as he has been refused leave to appeal against the impugned judgment of the High Court. The sole issue that remains to be examined in these appeals is regarding the companyrectness of the companyviction of the appellants for offence under Section 201 IPC. The Sessions Court has found that when Harbans Singh with PW8  Darshan Singh  reached the cremation ground, pyre of Manmohan Kaur was burning and all the accused along with many other persons were present there and according to the accused persons, dead body was in their house before it was taken to the cremation ground and cremated and also that numberautopsy on the dead body of Manmohan Kaur was companyducted. On these findings only, the companyclusion reached by the Sessions Court is that the body was cremated in a haste without informing the parents of the deceased and the Police and these circumstances indicate that all the accused persons being close relations and being in the same house had the knowledge or were having reasons to believe that offence in relation to the death of Manmohan Kaur had been companymitted and, thus, they caused the disappearance of the evidence of the crime of offence by cremating the dead body of Manmohan Kaur. The reasons for companyfirming the companyviction of the appellants stated by the High Court are that the accused persons, except Harjap Singh who was a close friend of Baldev Singh, are family members of Baldev Singh and they knew about number only the death of Manmohan Kaur but also the fact that she was missing from the house of Baldev Singh. It has been further numbericed that from the statement of Harbans Singh  PW4 , Kulwaran Singh  PW5  and Darshan Singh  PW8 , it appears that all the accused were present at the cremation ground when dead body of Manmohan Kaur was cremated and, thus, they were responsible for eliminating the evidence by getting the dead body cremated without informing the police as also Harbans Singh. They all acted in prosecution of companymon object in getting the evidence of the offence eliminated and in screening Baldev Singh of offence punishable under Section 306 IPC. Baldev Singh has been companyvicted and the appellants acquitted of offence under Section 306 IPC, namely, abetment of suicide. The appellants have been found guilty of offence under Section 201 IPC. For companyviction under the said offence, the prosecution was required to prove that the appellants had knowledge or had reason to believe that an offence under Section 306 had been companymitted by Baldev Singh and with such knowledge or belief they caused evidence of companymission of the offence to disappear either with the intention of screening the offender from legal punishment or with that intention gave any information respecting the offence which they knew or believed to be false. The only finding recorded against the appellants is that they are family members of Baldev Singh. In respect of Harpal Singh, only finding is that he was a family friend. Further finding recorded is that they were present at the house where the body of Manmohan Kaur had been kept and also at the cremation ground. The mere presence of the accused at the house or at the cremation ground or their relationship with her husband would number attract the provision of Section 201 IPC. Presumption that the appellants had the knowledge of companymission of offence cannot be drawn from their mere presence at the house or cremation ground or on account of relationship. There is numberother finding except above numbericed against the appellants. We have also perused the record. There is numberevidence to prove the guilt of the appellants for offence under Section 201 IPC. It cannot be held that the appellants knew or had reason to believe that offence had been companymitted and participated in cremation to companyceal and dispose of the dead body. In absence of evidence, it cannot be assumed on suspicion alone that the appellants must have known or must have reason to believe that Baldev Singh abetted in companymission of offence and, by being present at the cremation ground, they caused the evidence of companymission of the offence to disappear with intention to screen Baldev Singh from legal punishment. This Court in Palvinder Kaur v. The State of Punjab  1953  4 SCR 94 has held that in order to establish the charge under Section 201 IPC, it is essential to prove that an offence has been companymitted mere suspicion that it has been companymitted is number sufficient. It has to be proved that the accused knew or had reason to believe that such offence had been companymitted, and with the requisite knowledge and with the intent to screen the offender from legal punishment caused the evidence thereof to disappear or gave false information respecting such offence knowing or having reason to believe the same to be false. Palvinder Kaurs decision has been followed in various later decisions Suleman Rahiman Mulani Anr. v. State of Maharashtra AIR 1968 SC 829 Nathu Anr. v. State of Uttar Pradesh  1979  3 SCC 574 and V.L. Tresa v. State of Kerala  2001  3 SCC 549."

In [ ]:
print("—————————— Opening statements ——————————\n")
history: List[Dict[str, str]] = []

p = prosecution.respond(f"Make a short opening statement to the court. It should be an overview. Do not exhaust all your facts. Background: {case_background}")
print(p, "\n")

pl = plaintiff.respond("Make a very short opening statement no longer than 50 words in length.")
print(pl, "\n")

d = defense.respond(f"Make a short opening statement to the court. It should be an overview. Do not exhaust all your facts.")
print(d, "\n")

de = defendant.respond(f"Make a very short opening statement no longer than 50 words in length.")
print(de, "\n")

j = judge.respond(f"Review the opening statements given by the prosecution and the defendant. DO NOT MAKE A VERDICT.")
print(j, "\n")

round = 1
witcount = 1

#Loop governing main body of the proceedings
while round <= 4:
  print(f"—————————— Round {round} ——————————\n")

  p = prosecution.respond("Structure your argument to prove that the defendant is guilty. Analyse the EXACT previous response given by the defense (given in your prompt) and refute their points. Derive your understanding of the defense's argument SOLELY from the prompt given to you. Do not make up the defense's words on your own. If you wish to question a witness, insert the string 'I HEREBY SUMMON A WITNESS.' and ask the question you would like the witness to answer. If you do not wish to call a witness, skip this step. Use the string 'I HEREBY SUMMON A WITNESS.' EXACTLY, do NOT modify it. Also, you do not have to call a witness if you do not wish to.")
  print(p, "\n")

  #Checks if a witness was summoned by the prosecution
  if "I HEREBY SUMMON A WITNESS" in p.upper():
    WITNESSNEW = WITNESS.replace("$", str(witcount)) #Updates the display name of the witness with the witness number.
    witness = Agent("Witness", WITNESSNEW)
    w = witness.respond("Testify in no longer than 75 words. Only answer the question you have been asked.")
    print(w, "\n")
    witcount += 1

  pl = plaintiff.respond("Respond in no longer than 50 words.")
  print(pl, "\n")

  d = defense.respond("Structure your argument to prove that the defendant is innocent. Analyse the EXACT previous response given by the prosecution (given in your prompt) and refute their points. Derive your understanding of the prosecution's argument SOLELY from the prompt given to you. Do not make up the prosecution's words on your own. If you wish to question a witness, insert the string 'I HEREBY SUMMON A WITNESS.' and ask the question you would like the witness to answer. If you do not wish to call a witness, skip this step. Use the string 'I HEREBY SUMMON A WITNESS.' EXACTLY, do NOT modify it. Also, you do not have to call a witness if you do not wish to.")
  print(d, "\n")

  #Checks if a witness was summoned by the defense.
  if "I HEREBY SUMMON A WITNESS" in d.upper():
    WITNESSNEW = WITNESS.replace("$", str(witcount))
    witness = Agent("Witness", WITNESSNEW)
    w = witness.respond("Testify in no longer than 75 words. Only answer the question you have been asked.")
    print(w, "\n")
    witcount += 1

  de = defendant.respond(f"Respond in no longer than 50 words.")
  print(de, "\n")

  #Adjourns the court if round 4 is reached
  if round == 4:
    j = judge.respond("Evaluate the arguments of both sides and make a verdict. You must end the case here. If you cannot decide whether the defendant is guilty or not, resume the case on a later date.")
    print(j, "\n")
  else:
    j = judge.respond("Evaluate the arguments of both sides. If there is enough ground to make a verdict, give a verdict. If it remains unclear, insert the string 'ANOTHER ROUND.' at the VERY END of your response. Do NOT insert 'ANOTHER ROUND.' if you have given a verdict. Only give a verdict if it is absolutely clear whether the defendant is guilty or not. Else, call for another round.")
    print(j, "\n")

  #Checks if the judge has called for another round
  if 'ANOTHER ROUND' in j.upper():
    round += 1
  else:
    break
print("——————————————————————————————")